In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

     |████████████████████████████████| 450kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 46.3MB/s eta 0:00:01
     |████████████████████████████████| 122kB 57.6MB/s eta 0:00:01
     |████████████████████████████████| 235kB 47.5MB/s eta 0:00:01
     |████████████████████████████████| 409kB 50.9MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 52.8MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 26.8MB/s eta 0:00:01
     |████████████████████████████████| 41.0MB 3.2MB/s eta 0:00:011
     |████████████████████████████████| 51kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 2.8MB/s eta 0:00:01

In [ ]:
# !pip install snowflake-connector-python
!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

In [1]:
from snowflake.snowpark import Session

In [2]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

In [5]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip list | grep -i cloud

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
df = registry.show_models()
df
# type(df)

# Get model list from snowpark registry

In [ ]:
model_list = registry.models()
model_list

# Get model details from registry

In [ ]:
m =registry.get_model("my_model")
m

# Get model versions

In [ ]:
df_version = m.show_versions()
df_version

# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [ ]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(x_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

In [ ]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [ ]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

In [ ]:
mv = registry.log_model(gbm_default,
                   model_name="my_model",
                   version_name="v2",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
#                     python_version="3.9",
#                    metrics={"score": 96},
                   sample_input_data=x_train)

In [ ]:
def pickle_dumps(obj, file_path):
    import cloudpickle
    out = open(file_path, "wb")
    cloudpickle.dump(obj, out)
    out.close()

In [ ]:
pickle_dumps(gbm_default, "/notebooks/notebooks/ml_model_1")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [6]:
# To see the stages
session.sql("show stages").show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"     |"database_name"           |"schema_name"  |"url"  |"has_credentials"  |"has_encryption_key"  |"owner"  |"comment"  |"region"  |"type"    |"cloud"  |"notification_channel"  |"storage_integration"  |"endpoint"  |"owner_role_type"  |"directory_enabled"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-05-23 05:38:13.276000-07:00  |ML_MODELS  |MODEL_REGISTRY_SNOWSIGHT  |SNOWSIGHT      |      

# Upload model file on ML_MODELS stage 

In [ ]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE ML_MODELS').show()

In [8]:
session.file.put(
    "/notebooks/notebooks/ml_model_2", "@ML_MODELS", auto_compress=False, overwrite=True
)

[PutResult(source='ml_model_2', target='ml_model_2', source_size=129477, target_size=129488, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

# Read model file from stage

In [9]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F


def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'get_model_from_file', is_permanent = True, replace = True, stage_location = '@ML_MODELS')
def get_model_from_file(data_list: list) -> list:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file('ml_model_2')
    
    return "test"
#     return pipeline.predict(data_array).to_list()

# 